In [1]:
from google.colab import files
files.upload()  # kaggle.json 업로드 창 표시
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
from pathlib import Path

In [3]:
root = Path('./Road_Sign_dataset')
Path('./Road_Sign_dataset/train').mkdir(parents=True, exist_ok=True)
Path('./Road_Sign_dataset/val').mkdir(parents=True, exist_ok=True)
Path('./Road_Sign_dataset/test').mkdir(parents=True, exist_ok=True)

In [4]:
for path1 in ('images', 'labels'):
    for path2 in ('train', 'val', 'test'):
        new_path = root / 'Sign' / path1 / path2
        new_path.mkdir(parents=True, exist_ok=True)

In [5]:
!kaggle datasets download andrewmvd/road-sign-detection

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/road-sign-detection
License(s): CC0-1.0
 76% 166M/218M [00:00<00:00, 1.74GB/s]
100% 218M/218M [00:00<00:00, 1.73GB/s]


In [6]:
!unzip -q road-sign-detection.zip

In [7]:
import os
import glob
import shutil
import random

In [8]:
src_xml_dir = '/content/annotations'
src_img_dir = '/content/images'

base_dest_dir = '/content/Road_Sign_dataset/Sign'

# 이미 만들어두신 폴더 구조에 맞게 경로 설정
dest_dirs = {
    'train': {
        'images': os.path.join(base_dest_dir, 'images/train'),
        'labels': os.path.join(base_dest_dir, 'labels/train')
    },
    'val': {
        'images': os.path.join(base_dest_dir, 'images/val'),
        'labels': os.path.join(base_dest_dir, 'labels/val')
    },
    'test': {
        'images': os.path.join(base_dest_dir, 'images/test'),
        'labels': os.path.join(base_dest_dir, 'labels/test')
    }
}

In [9]:
xml_files = glob.glob(os.path.join(src_xml_dir, '*.xml'))
file_ids = [Path(f).stem for f in xml_files]

random.seed(2026)
random.shuffle(file_ids)

In [10]:
total_files = len(file_ids)
train_idx = int(total_files * 0.6)
val_idx = int(total_files * 0.2)

In [11]:
train_ids = file_ids[:train_idx]
val_ids = file_ids[train_idx : train_idx + val_idx]
test_ids = file_ids[train_idx + val_idx:]

In [12]:
print(f"Total files: {total_files}")
print(f"Train: {len(train_ids)}, Val: {len(val_ids)}, Test: {len(test_ids)}")

Total files: 877
Train: 526, Val: 175, Test: 176


In [13]:
import xml.etree.ElementTree as ET

# 1. XML 파일들이 있는 경로 (원본 경로 사용)
xml_dir = '/content/annotations'

# 2. 클래스 이름을 담을 집합(Set) 생성 (중복 제거를 위해)
classes = set()

# 3. 모든 XML 파일을 순회하며 클래스 이름 추출
xml_files = glob.glob(os.path.join(xml_dir, '*.xml'))

print(f"총 {len(xml_files)}개의 XML 파일을 분석합니다...")

for xml_file in xml_files:
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # <object> 태그 내의 <name> 태그 값을 찾음
        for obj in root.findall('object'):
            name = obj.find('name').text
            classes.add(name)
    except Exception as e:
        print(f"Error parsing {xml_file}: {e}")

# 4. 리스트로 변환 및 정렬 (알파벳 순 등 원하는 순서가 있다면 직접 지정 필요)
# YOLO 학습 시 클래스 순서(ID)가 중요하므로, 보통은 정렬해서 고정합니다.
sorted_classes = sorted(list(classes))

# 5. voc.names 파일 작성
output_file = 'voc.names'
with open(output_file, 'w') as f:
    for cls in sorted_classes:
        f.write(cls + '\n')

print(f"\n[완료] '{output_file}' 파일이 생성되었습니다.")
print(f"발견된 클래스 개수: {len(sorted_classes)}")
print(f"클래스 목록: {sorted_classes}")

총 877개의 XML 파일을 분석합니다...

[완료] 'voc.names' 파일이 생성되었습니다.
발견된 클래스 개수: 4
클래스 목록: ['crosswalk', 'speedlimit', 'stop', 'trafficlight']


In [14]:
def copy_dataset(ids, split_name):
    print(f"\nProcessing {split_name} data...")

    dest_img_path = dest_dirs[split_name]['images']
    dest_lbl_path = dest_dirs[split_name]['labels']

    os.makedirs(dest_img_path, exist_ok=True)
    os.makedirs(dest_lbl_path, exist_ok=True)

    for file_id in ids:
        src_xml = os.path.join(src_xml_dir, file_id + '.xml')
        shutil.move(src_xml, os.path.join(dest_lbl_path, file_id + '.xml'))

        found_image = False
        for ext in ['.jpg', '.png', '.jpeg', '.JPG']:
            src_img = os.path.join(src_img_dir, file_id + ext)
            if os.path.exists(src_img):
                shutil.move(src_img, os.path.join(dest_img_path, file_id + ext))
                found_image = True
                break

        if not found_image:
            print(f"Warning: Image for {file_id} not found.")

copy_dataset(train_ids, 'train')
copy_dataset(val_ids, 'val')
copy_dataset(test_ids, 'test')

print("파일 이동 완료")


Processing train data...

Processing val data...

Processing test data...
파일 이동 완료


In [15]:
%cd /content/Road_Sign_dataset

/content/Road_Sign_dataset


In [16]:
!git clone https://github.com/ssaru/convert2Yolo.git

Cloning into 'convert2Yolo'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 215 (delta 38), reused 35 (delta 35), pack-reused 172 (from 1)
Receiving objects: 100% (215/215), 994.67 KiB | 8.02 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [17]:
%cd /content/Road_Sign_dataset/convert2Yolo

/content/Road_Sign_dataset/convert2Yolo


In [18]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 60.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 9.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [21]:
!python example.py \
    --datasets VOC \
    --img_path /content/Road_Sign_dataset/Sign/images/train \
    --label /content/Road_Sign_dataset/Sign/labels/train \
    --convert_output_path /content/Road_Sign_dataset/Sign/labels/train \
    --img_type '.png' \
    --manifest_path /content/Road_Sign_dataset/Sign \
    --cls_list_file ./voc.names


VOC Parsing:   |████████████████████████████████████████| 100.0% (526/526)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (526/526)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (526/526)  Complete



In [22]:
!python example.py \
    --datasets VOC \
    --img_path /content/Road_Sign_dataset/Sign/images/val \
    --label /content/Road_Sign_dataset/Sign/labels/val \
    --convert_output_path /content/Road_Sign_dataset/Sign/labels/val \
    --img_type '.png' \
    --manifest_path /content/Road_Sign_dataset/Sign \
    --cls_list_file ./voc.names


VOC Parsing:   |████████████████████████████████████████| 100.0% (175/175)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (175/175)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (175/175)  Complete



In [23]:
!python example.py \
    --datasets VOC \
    --img_path /content/Road_Sign_dataset/Sign/images/test \
    --label /content/Road_Sign_dataset/Sign/labels/test \
    --convert_output_path /content/Road_Sign_dataset/Sign/labels/test \
    --img_type '.png' \
    --manifest_path /content/Road_Sign_dataset/Sign \
    --cls_list_file ./voc.names


VOC Parsing:   |████████████████████████████████████████| 100.0% (176/176)  Complete


YOLO Generating:|████████████████████████████████████████| 100.0% (176/176)  Complete


YOLO Saving:   |████████████████████████████████████████| 100.0% (176/176)  Complete



In [24]:
%cd /content/

/content


In [25]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.3 MB/s eta 0:00:00


In [26]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [27]:
model = YOLO('yolov8s.pt')

In [29]:
# yaml 파일 생성
import yaml

# 1. voc.names에서 클래스 이름 읽어오기
with open('/content/Road_Sign_dataset/convert2Yolo/voc.names', 'r') as f:
    # 빈 줄 제거하고 이름만 리스트로 만듦
    classes = [line.strip() for line in f.readlines() if line.strip()]

# 2. YOLO 학습을 위한 YAML 데이터 정의
# path: 데이터셋의 루트 경로 (절대 경로)
# train, val, test: path 기준 상대 경로
data = {
    'path': '/content/Road_Sign_dataset/Sign',
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'names': {i: name for i, name in enumerate(classes)}  # {0: 'car', 1: 'bus'...} 형태
}

# 3. yaml 파일로 저장
yaml_path = '/content/Road_Sign_dataset/Sign/sign_data.yaml'

with open(yaml_path, 'w') as f:
    yaml.dump(data, f, sort_keys=False)

print(f"설정 파일이 생성되었습니다: {yaml_path}")
print("내용 확인:")
print(data)

설정 파일이 생성되었습니다: /content/Road_Sign_dataset/Sign/sign_data.yaml
내용 확인:
{'path': '/content/Road_Sign_dataset/Sign', 'train': 'images/train', 'val': 'images/val', 'test': 'images/test', 'names': {0: 'crosswalk', 1: 'speedlimit', 2: 'stop', 3: 'trafficlight'}}


In [30]:
results = model.train(
    data='/content/Road_Sign_dataset/Sign/sign_data.yaml',
    epochs=10,
    batch=32,
    imgsz=640,
    device=0,
    workers=2,
    name='custom_s'
)

Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Road_Sign_dataset/Sign/sign_data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=custom_s, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, pers

In [31]:
# 학습된 모델 로드
model = YOLO("/content/runs/detect/custom_s/weights/best.pt")

# 검증 수행
results = model.val(
    data="/content/Road_Sign_dataset/Sign/sign_data.yaml",  # 데이터셋 설정
    imgsz=640,               # 이미지 크기
    iou=0.5,                 # IoU 임계값
    batch=32,                # 배치 크기
    device=0,                # GPU 사용
    workers=2,               # 데이터 로드 시 병렬 처리할 워커 수
    half=True,               # FP16 연산 활성화 (속도 향상)
    split="val"
)

print(results)

Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2665.7±1116.2 MB/s, size: 206.0 KB)
val: Scanning /content/Road_Sign_dataset/Sign/labels/val.cache... 175 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 175/175 355.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 2.9it/s 2.0s
                   all        175        245      0.937      0.884      0.904      0.748
             crosswalk         33         42      0.929      0.881      0.937      0.711
            speedlimit        139        159       0.97          1      0.991      0.878
                  stop         15         15      0.908          1       0.97      0.932
          trafficlight         17         29      0.941      0.655      0.717       0.47
Speed: 1.3ms preproc

In [33]:
# 학습된 모델 로드
model = YOLO("/content/runs/detect/custom_s/weights/best.pt")

# 테스트 수행
results = model.val(
    data="/content/Road_Sign_dataset/Sign/sign_data.yaml",  # 데이터셋 설정
    imgsz=640,               # 이미지 크기
    iou=0.5,                 # IoU 임계값
    batch=32,                # 배치 크기
    device=0,                # GPU 사용
    workers=2,               # 데이터 로드 시 병렬 처리할 워커 수
    half=True,               # FP16 연산 활성화 (속도 향상)
    split="test"
)

print(results)

Ultralytics 8.3.248 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2751.6±1315.4 MB/s, size: 215.0 KB)
val: Scanning /content/Road_Sign_dataset/Sign/labels/test... 176 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 176/176 828.0it/s 0.2s
val: New cache created: /content/Road_Sign_dataset/Sign/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 3.1it/s 1.9s
                   all        176        251      0.918      0.933      0.961      0.797
             crosswalk         38         44      0.861      0.909      0.938      0.744
            speedlimit        140        160      0.979      0.994      0.995      0.875
                  stop         18         18      0.834          1      0.982      0.861
          trafficlight         18        

In [34]:
import pandas as pd

# --- 1. 전체 종합 점수 (Highlight) ---
print("\n" + "="*40)
print(f"   🏆 최종 테스트 성능 요약 (Test Set)")
print("="*40)
print(f" • mAP@50-95 (종합 점수) : {results.box.map:.4f}")  # 0~1 사이, 높을수록 좋음
print(f" • mAP@50    (탐지 능력) : {results.box.map50:.4f}") # IoU 0.5 기준 점수
print(f" • Precision (정밀도)    : {results.box.mp:.4f}")   # 오탐지(False Positive)가 적은지
print(f" • Recall    (재현율)    : {results.box.mr:.4f}")   # 놓친 것(False Negative)이 적은지
print("="*40 + "\n")


# --- 2. 클래스별 상세 점수 (DataFrame 활용) ---
print("🔍 [클래스별 상세 성능 (mAP@50-95 기준)]")

# 클래스 이름과 해당 클래스의 mAP 점수 가져오기
class_ids = results.names.keys()
class_names = [results.names[i] for i in class_ids]
class_maps = results.box.maps  # 각 클래스별 mAP50-95 배열

df = pd.DataFrame({
    'Class Name': class_names,
    'mAP': class_maps
})

# 점수가 높은 순서대로 정렬해서 출력
df_sorted = df.sort_values(by='mAP', ascending=False).reset_index(drop=True)

# 표 출력
print(df_sorted.to_string(formatters={'mAP': '{:.4f}'.format}))


   🏆 최종 테스트 성능 요약 (Test Set)
 • mAP@50-95 (종합 점수) : 0.7971
 • mAP@50    (탐지 능력) : 0.9609
 • Precision (정밀도)    : 0.9181
 • Recall    (재현율)    : 0.9326

🔍 [클래스별 상세 성능 (mAP@50-95 기준)]
     Class Name    mAP
0    speedlimit 0.8750
1          stop 0.8609
2     crosswalk 0.7435
3  trafficlight 0.7087


In [35]:
model = YOLO("/content/runs/detect/custom_s/weights/best.pt")

results = model.predict(
    source="/content/Road_Sign_dataset/Sign/images/custom",  # 테스트 이미지 폴더
    imgsz=640,           # 입력 이미지 크기
    conf=0.25,           # 신뢰도(Confidence) 임계값
    device=0,            # GPU 사용 (CPU 사용 시 "cpu")
    save=True,           # 탐지 결과 저장
    save_txt=True,       # 탐지 결과를 txt 형식으로 저장 (YOLO 포맷)
    save_conf=True       # 탐지된 객체의 신뢰도 점수도 저장
)

print(results)


image 1/16 /content/Road_Sign_dataset/Sign/images/custom/chile-693053_640.jpg: 384x640 1 speedlimit, 70.2ms
image 2/16 /content/Road_Sign_dataset/Sign/images/custom/istockphoto-171579151-1024x1024.jpg: 448x640 1 speedlimit, 70.0ms
image 3/16 /content/Road_Sign_dataset/Sign/images/custom/outback-7570648_1280.jpg: 640x448 1 stop, 69.8ms
image 4/16 /content/Road_Sign_dataset/Sign/images/custom/pedestrian-crossing-3906_640.jpg: 480x640 1 crosswalk, 70.8ms
image 5/16 /content/Road_Sign_dataset/Sign/images/custom/road-sign-325400_640.jpg: 448x640 1 crosswalk, 8.3ms
image 6/16 /content/Road_Sign_dataset/Sign/images/custom/road106.png: 640x448 1 speedlimit, 7.6ms
image 7/16 /content/Road_Sign_dataset/Sign/images/custom/route-61-510231_640.jpg: 448x640 (no detections), 7.6ms
image 8/16 /content/Road_Sign_dataset/Sign/images/custom/rural-3875466_640.jpg: 448x640 (no detections), 7.0ms
image 9/16 /content/Road_Sign_dataset/Sign/images/custom/sign-353741_640.jpg: 480x640 1 crosswalk, 7.6ms
image 

In [37]:
for result in results:
    boxes = result.boxes  # 탐지된 박스 정보

    # 몇 개가 탐지되었는지 간략히 출력
    print(f"이미지: {result.path}")
    print(f"탐지된 객체 수: {len(boxes)}")
    print("-" * 20)

이미지: /content/Road_Sign_dataset/Sign/images/custom/chile-693053_640.jpg
탐지된 객체 수: 1
--------------------
이미지: /content/Road_Sign_dataset/Sign/images/custom/istockphoto-171579151-1024x1024.jpg
탐지된 객체 수: 1
--------------------
이미지: /content/Road_Sign_dataset/Sign/images/custom/outback-7570648_1280.jpg
탐지된 객체 수: 1
--------------------
이미지: /content/Road_Sign_dataset/Sign/images/custom/pedestrian-crossing-3906_640.jpg
탐지된 객체 수: 1
--------------------
이미지: /content/Road_Sign_dataset/Sign/images/custom/road-sign-325400_640.jpg
탐지된 객체 수: 1
--------------------
이미지: /content/Road_Sign_dataset/Sign/images/custom/road106.png
탐지된 객체 수: 1
--------------------
이미지: /content/Road_Sign_dataset/Sign/images/custom/route-61-510231_640.jpg
탐지된 객체 수: 0
--------------------
이미지: /content/Road_Sign_dataset/Sign/images/custom/rural-3875466_640.jpg
탐지된 객체 수: 0
--------------------
이미지: /content/Road_Sign_dataset/Sign/images/custom/sign-353741_640.jpg
탐지된 객체 수: 1
--------------------
이미지: /content/Road_Sign_da

In [40]:
import matplotlib.pyplot as plt
import cv2
import math

In [41]:
# 1. 설정
n_cols = 4                     # 한 줄에 4개씩
n_images = len(results)        # 전체 이미지 개수
n_rows = math.ceil(n_images / n_cols)  # 필요한 행(줄) 개수 계산

# 2. 전체 캔버스 크기 설정 (가로 20, 세로는 행 개수에 비례해서 늘림)
plt.figure(figsize=(20, 5 * n_rows))

# 3. 반복문으로 하나씩 그리기
for i, r in enumerate(results):
    # 그리드 위치 지정 (행, 열, 몇 번째인지)
    plt.subplot(n_rows, n_cols, i + 1)

    # 이미지 변환 및 가져오기
    im_bgr = r.plot()
    im_rgb = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)

    # 출력
    plt.imshow(im_rgb)
    plt.axis('off')             # 축 정보 제거
    plt.title(f"Result {i+1}")  # 제목 (선택)

plt.tight_layout() # 간격 자동 조절
plt.show()

Output hidden; open in https://colab.research.google.com to view.